CONTEXT BASED RECOMMENDER SYSTEM


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [3]:
from scipy.sparse import csr_matrix


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
song_data = pd.read_csv('ex.csv')


In [9]:
song_data.head(10)



,Song-Name,Singer/Artists,Genre,Album/Movie,User-Rating
0,Aankh Marey,"Kumar Sanu, Mika Singh, Neha Kakkar",BollywoodDance,Simmba,8.8/10
1,Coca Cola,"Neha Kakkar, Tony Kakkar",BollywoodDanceRomantic,Luka Chuppi,9.0/10
2,Apna Time Aayega,Ranveer Singh,BollywoodDance,Gully Boy,9.7/10
3,Mungda,"Jyotica Tangri, Shaan, Subhro Ganguly",BollywoodDance,Total Dhamaal,9.1/10
4,Tere Bin,"Asees Kaur, Rahat Fateh Ali Khan, Tanishk Bagchi",BollywoodRomantic,Simmba,9.2/10
5,Gali Gali,Neha Kakkar,BollywoodDance,KGF,9.1/10
6,Chamma Chamma,"Arun, Ikka, Neha Kakkar, Romy",BollywoodDance,Fraud Saiyaan,9.2/10
7,Mere Gully Mein,Ranveer Singh,BollywoodDance,Gully Boy,9.5/10
8,Kamariya,Darshan Raval,BollywoodDance,Mitron,9.2/10
9,Ek Ladki Ko Dekha Toh Aisa Laga,"Darshan Raval, Rochak Kohli",BollywoodRomantic,Ek Ladki Ko Dekha Toh Aisa Laga,9.6/10


**bold text**

In [12]:
song_data.describe()

,Song-Name,Singer/Artists,Genre,Album/Movie,User-Rating
count,2420,2410,2420,2417,2420
unique,2337,1214,21,902,40
top,Naina,Mohammed Rafi,BollywoodDance,Devdas,9.4/10
freq,3,55,1228,10,281


In [14]:
song_data.isnull().sum()

Song-Name          0
Singer/Artists    10
Genre              0
Album/Movie        3
User-Rating        0
dtype: int64

In [16]:
song_data=song_data.drop_duplicates()

In [18]:
song_data.dropna(inplace=True)

In [20]:
song_data.isnull().sum()

Song-Name         0
Singer/Artists    0
Genre             0
Album/Movie       0
User-Rating       0
dtype: int64

In [22]:
print(f"There are {song_data.shape[0]} observations in the dataset")

There are 2391 observations in the dataset


In [24]:
#Unique songs
unique_songs = song_data['Song-Name'].unique().shape[0]
print(f"There are {unique_songs} unique songs in the dataset")

There are 2327 unique songs in the dataset


In [26]:
# Preprocess and combine features
song_data['combined_features'] = song_data['Song-Name'] + ' ' + song_data['Singer/Artists'] + ' ' + song_data['Genre'] + ' ' + song_data['Album/Movie']

# Create a mapping from song names to indices
song_name_to_index = pd.Series(song_data.index, index=song_data['Song-Name']).to_dict()

# Vectorize combined features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(song_data['combined_features'])

In [28]:
from sklearn.neighbors import NearestNeighbors
from rapidfuzz import fuzz
import numpy as np

class Recommender:
    def __init__(self, metric='cosine', algorithm='brute', k=5, data=None, decode_id_song=None):
        self.metric = metric
        self.algorithm = algorithm
        self.k = k
        self.data = data
        self.decode_id_song = decode_id_song
        self.model = self._recommender().fit(data)
    
    def make_recommendation(self, new_song, n_recommendations):
        recommended = self._recommend(new_song=new_song, n_recommendations=n_recommendations)
        print("... Done")
        return recommended 
    
    def _recommender(self):
        return NearestNeighbors(metric=self.metric, algorithm=self.algorithm, n_neighbors=self.k, n_jobs=-1)
    
    def _recommend(self, new_song, n_recommendations):
        recommendations = []
        recommendation_ids = self._get_recommendations(new_song=new_song, n_recommendations=n_recommendations)
        recommendations_map = self._map_indices_to_song_title(recommendation_ids)
        for i, (idx, dist) in enumerate(recommendation_ids):
            recommendations.append(recommendations_map[idx])
        return recommendations
    
    def _get_recommendations(self, new_song, n_recommendations):
        recom_song_id = self._fuzzy_matching(song=new_song)
        print(f"Starting the recommendation process for {new_song} ...")
        distances, indices = self.model.kneighbors(self.data[recom_song_id], n_neighbors=n_recommendations+1)
        return sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    
    def _map_indices_to_song_title(self, recommendation_ids):
        return {song_id: song_title for song_title, song_id in self.decode_id_song.items()}
    
    def _fuzzy_matching(self, song):
        match_tuple = []
        for title, idx in self.decode_id_song.items():
            ratio = fuzz.ratio(title.lower(), song.lower())
            if ratio >= 60:
                match_tuple.append((title, idx, ratio))
        match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
        if not match_tuple:
            print(f"The recommendation system could not find a match for {song}")
            return None
        return match_tuple[0][1]


In [29]:
from sklearn.neighbors import NearestNeighbors
from rapidfuzz import fuzz
import numpy as np

class Recommender:
    def __init__(self, metric='cosine', algorithm='brute', k=5, data=None, decode_id_song=None):
        self.metric = metric
        self.algorithm = algorithm
        self.k = k
        self.data = data
        self.decode_id_song = decode_id_song
        self.model = self._recommender().fit(data)
    
    def make_recommendation(self, new_song, n_recommendations):
        recommended = self._recommend(new_song=new_song, n_recommendations=n_recommendations)
        print("... Done")
        return recommended 
    
    def _recommender(self):
        return NearestNeighbors(metric=self.metric, algorithm=self.algorithm, n_neighbors=self.k, n_jobs=-1)
    
    def _recommend(self, new_song, n_recommendations):
        recommendations = []
        recommendation_ids = self._get_recommendations(new_song=new_song, n_recommendations=n_recommendations)
        recommendations_map = self._map_indices_to_song_title(recommendation_ids)
        for i, (idx, dist) in enumerate(recommendation_ids):
            recommendations.append(recommendations_map[idx])
        return recommendations
    
    def _get_recommendations(self, new_song, n_recommendations):
        recom_song_id = self._fuzzy_matching(song=new_song)
        print(f"Starting the recommendation process for {new_song} ...")
        distances, indices = self.model.kneighbors(self.data[recom_song_id], n_neighbors=n_recommendations+1)
        return sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    
    def _map_indices_to_song_title(self, recommendation_ids):
        return {song_id: song_title for song_title, song_id in self.decode_id_song.items()}
    
    def _fuzzy_matching(self, song):
        match_tuple = []
        for title, idx in self.decode_id_song.items():
            ratio = fuzz.ratio(title.lower(), song.lower())
            if ratio >= 60:
                match_tuple.append((title, idx, ratio))
        match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
        if not match_tuple:
            print(f"The recommendation system could not find a match for {song}")
            return None
        return match_tuple[0][1]


In [32]:
# Initialize the recommender with the prepared data
recommender = Recommender(
    metric='cosine',
    algorithm='brute',
    k=5,
    data=X,
    decode_id_song=song_name_to_index
)

# Example recommendation
recommendations = recommender.make_recommendation('Some Song Name', 7)
print(recommendations)


Starting the recommendation process for Some Song Name ...
... Done
['Raat Ka Nasha', 'Dekha Tujhe Toh', 'Falak Dekhun', 'Dulhan Hum Le Jayenge', 'Jaaneman Jaane Jaan', 'Brothers Mashup', 'Dil Tham Chale Hum Aaj Kidhar']


In [34]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Load data
data = pd.read_csv('ex.csv')



In [36]:
print(data.isnull().sum())

Song-Name          0
Singer/Artists    10
Genre              0
Album/Movie        3
User-Rating        0
dtype: int64


In [38]:
data.dropna(inplace=True)

In [40]:
data.isnull().sum()

Song-Name         0
Singer/Artists    0
Genre             0
Album/Movie       0
User-Rating       0
dtype: int64

In [42]:
# Convert 'User-Rating' to numeric after stripping the '/10' part and converting to float
def convert_rating(rating):
    try:
        return float(rating.split('/')[0])
    except ValueError:
        return None

data['User-Rating'] = data['User-Rating'].apply(convert_rating)

# Pivot the table to create a user-item matrix
user_item_matrix = data.pivot_table(index='Song-Name', columns='Singer/Artists', values='User-Rating')

# Fill NaN values with 0, indicating that the user hasn't rated the song
user_item_matrix = user_item_matrix.fillna(0)


In [44]:
data.head()

,Song-Name,Singer/Artists,Genre,Album/Movie,User-Rating
0,Aankh Marey,"Kumar Sanu, Mika Singh, Neha Kakkar",BollywoodDance,Simmba,8.8
1,Coca Cola,"Neha Kakkar, Tony Kakkar",BollywoodDanceRomantic,Luka Chuppi,9.0
2,Apna Time Aayega,Ranveer Singh,BollywoodDance,Gully Boy,9.7
3,Mungda,"Jyotica Tangri, Shaan, Subhro Ganguly",BollywoodDance,Total Dhamaal,9.1
4,Tere Bin,"Asees Kaur, Rahat Fateh Ali Khan, Tanishk Bagchi",BollywoodRomantic,Simmba,9.2


In [46]:
# Pivot the table to create a user-item matrix
user_item_matrix = data.pivot_table(index='Song-Name', columns='Singer/Artists', values='User-Rating')

# Fill NaN values with 0, indicating that the user hasn't rated the song
user_item_matrix = user_item_matrix.fillna(0)

# Standardize the data (important for KNN)
scaler = StandardScaler()
user_item_matrix_scaled = scaler.fit_transform(user_item_matrix)

# Print the transformed matrix
print(user_item_matrix_scaled[:5])


[[-0.03592295 -0.02073458 -0.02073458 ... -0.08824199 -0.02073458
  -0.0414897 ]
 [-0.03592295 -0.02073458 -0.02073458 ... -0.08824199 -0.02073458
  -0.0414897 ]
 [-0.03592295 -0.02073458 -0.02073458 ... -0.08824199 -0.02073458
  -0.0414897 ]
 [-0.03592295 -0.02073458 -0.02073458 ... -0.08824199 -0.02073458
  -0.0414897 ]
 [-0.03592295 -0.02073458 -0.02073458 ... -0.08824199 -0.02073458
  -0.0414897 ]]


In [48]:
from sklearn.neighbors import NearestNeighbors

# Fit the model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_item_matrix_scaled)


NearestNeighbors(algorithm='brute', metric='cosine')

In [50]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from rapidfuzz import fuzz
import numpy as np

In [52]:
# Convert the DataFrame into a pivot table for sparse matrix creation
df_songs_features = user_item_matrix

# Obtain a sparse matrix
mat_songs_features = csr_matrix(df_songs_features.values)


In [53]:
# Define the Recommender Class
class Recommender:
    def __init__(self, metric, algorithm, k, data, decode_id_song):
        self.metric = metric
        self.algorithm = algorithm
        self.k = k
        self.data = data
        self.decode_id_song = decode_id_song
        self.model = self._recommender().fit(data)
    
    def make_recommendation(self, new_song, n_recommendations):
        recommended = self._recommend(new_song=new_song, n_recommendations=n_recommendations)
        print("... Done")
        return recommended 
    
    def _recommender(self):
        return NearestNeighbors(metric=self.metric, algorithm=self.algorithm, n_neighbors=self.k, n_jobs=-1)
    
    def _recommend(self, new_song, n_recommendations):
        # Get the id of the recommended songs
        recommendations = []
        recommendation_ids = self._get_recommendations(new_song=new_song, n_recommendations=n_recommendations)
        # Return the name of the song using a mapping dictionary
        recommendations_map = self._map_indeces_to_song_title(recommendation_ids)
        # Translate this recommendations into the ranking of song titles recommended
        for i, (idx, dist) in enumerate(recommendation_ids):
            recommendations.append(recommendations_map[idx])
        return recommendations
                 
    def _get_recommendations(self, new_song, n_recommendations):
        # Get the id of the song according to the text
        recom_song_id = self._fuzzy_matching(song=new_song)
        if recom_song_id is None:
            return []
        # Start the recommendation process
        print(f"Starting the recommendation process for {new_song} ...")
        # Return the n neighbors for the song id
        distances, indices = self.model.kneighbors(self.data[recom_song_id], n_neighbors=n_recommendations+1)
        return sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    
    def _map_indeces_to_song_title(self, recommendation_ids):
        # Get reverse mapper
        return {song_id: song_title for song_title, song_id in self.decode_id_song.items()}
    
    def _fuzzy_matching(self, song):
        match_tuple = []
        # Get match
        for title, idx in self.decode_id_song.items():
            ratio = fuzz.ratio(title.lower(), song.lower())
            if ratio >= 60:
                match_tuple.append((title, idx, ratio))
        # Sort
        match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
        if not match_tuple:
            print(f"The recommendation system could not find a match for {song}")
            return None
        return match_tuple[0][1]


In [56]:
# Define your decode_id_song mapping
decode_id_song = {title: idx for idx, title in enumerate(user_item_matrix.index)}

# Initialize the recommender with the specified parameters
model = Recommender(metric='cosine', algorithm='brute', k=20, data=mat_songs_features, decode_id_song=decode_id_song)

def get_recommendations():
    # Input song title from the user
    song = input("Enter the song title for recommendations: ")

    # Get recommendations
    new_recommendations = model.make_recommendation(new_song=song, n_recommendations=10)

    # Print the recommendations
    print(f"The recommendations for '{song}' are:")
    print(new_recommendations)

def main():
    while True:
        get_recommendations()
        # Ask the user if they want to get recommendations for another song
        user_input = input("Do you want to get recommendations for another song? (yes/no): ").strip().lower()
        if user_input not in ['yes', 'y']:
            print("Exiting the recommendation system.")
            break

if __name__ == "__main__":
    main()


Enter the song title for recommendations:  coca cola


Starting the recommendation process for coca cola ...
... Done
The recommendations for 'coca cola' are:
['Neele Neele Ambar par', 'Nazrein Mili Dil Dhadka', 'Nazrein Milaana Nazrein Churaana', 'Nazm Nazm', 'Nazdeekiyan', 'Neela Aasman So Gaya', 'Sawan Aaya Hai (Unplugged Version)', 'Mohabbat Nasha Hai', 'Helicopter', 'Mile Ho Tum (Reprise)']


Do you want to get recommendations for another song? (yes/no):  yes
Enter the song title for recommendations:  cola


Starting the recommendation process for cola ...
... Done
The recommendations for 'cola' are:
['Neele Neele Ambar par', 'Nazrein Mili Dil Dhadka', 'Nazrein Milaana Nazrein Churaana', 'Nazm Nazm', 'Nazdeekiyan', 'Neela Aasman So Gaya', 'Sawan Aaya Hai (Unplugged Version)', 'Mohabbat Nasha Hai', 'Helicopter', 'Mile Ho Tum (Reprise)']


Do you want to get recommendations for another song? (yes/no):  yes
Enter the song title for recommendations:  history


The recommendation system could not find a match for history
... Done
The recommendations for 'history' are:
[]


Do you want to get recommendations for another song? (yes/no):  no


Exiting the recommendation system.


In [58]:
import pickle

In [62]:
import pickle
with open('recommender_model.pkl', 'wb') as file:
    pickle.dump(model, file)
